In [ ]:
import torch

def weighted_mean_pooling(embeddings, mask, cast=True, method_b=False, fp16=False):
    if fp16:
      assert (embeddings == embeddings.to(torch.float16)).all()
      embeddings = embeddings.to(torch.float16)
      embeddings = torch.cat([embeddings] * 100, dim=1)
      mask = torch.cat([mask] * 100, dim=1)
    if cast:
      in_dtype = embeddings.dtype
      # Make sure dtype is fp32 due to instability
      embeddings = embeddings.to(torch.float32)
    # Get weights of shape [bs, seq_len, hid_dim]
    weights = (
        torch.arange(
            start=1,
            end=embeddings.shape[1] + 1,
            dtype=embeddings.dtype,
            device=embeddings.device,
        )
        .unsqueeze(0)
        .unsqueeze(-1)
        .expand(embeddings.size())
    )
    # Add hid_dim
    loss_mask_expanded = (
        mask.unsqueeze(-1).expand(embeddings.size()).to(embeddings.dtype)
    )

    # x / (a + b) = x
    # 8 / (3+1) = 2 ; == 4 / 2

    # Perform weighted mean pooling across seq_len: bs, seq_len, hidden_dim -> bs, hidden_dim
    # WARNING: This is unstable in fp16, as it will lead to values > 65519 (fp16 max) hence inf
    # This could be rewritten to be more stable in fp16, but at the cost of less fraction precision
    # i.e. there will be more digits, which may be less stable in bf16 which has less fractions
    # Thus to be safe do it all in fp32
    if method_b:
      # Make weights smaller first to avoid shooting above fp16's inf max; 50 is an arbitrary value
      embeddings = torch.sum(embeddings * loss_mask_expanded * ((weights/50) / (torch.sum(weights/50, dim=1))), dim=1)
    else:
      sum_embeddings = torch.sum(embeddings * loss_mask_expanded * weights, dim=1)
      sum_mask = torch.sum(loss_mask_expanded * weights, dim=1)
      embeddings = sum_embeddings / sum_mask

    if cast is False:
      return embeddings
    return embeddings.to(in_dtype)


x = torch.tensor([[[ 0.0072, -0.0188,  0.0033, -0.0124, -0.0015, -0.0467, -0.0666,
           0.0522, -0.0173,  0.0136,  0.0657,  0.0352,  0.0666, -0.0305,
           0.0021, -0.0270,  0.0138, -0.0418, -0.0464,  0.0106, -0.0194,
          -0.0094, -0.0749, -0.0463, -0.0009,  0.0207, -0.0244,  0.0226,
           0.0014, -0.0093,  0.0035,  0.0293, -0.0367,  0.0552, -0.0272,
           0.0151,  0.0370,  0.0175,  0.0387, -0.0068,  0.0092, -0.0072,
           0.0075, -0.0275, -0.0258, -0.0112, -0.0266, -0.0077,  0.0140,
           0.0691,  0.0591, -0.0023,  0.0225, -0.0075, -0.0414,  0.0135,
           0.0245,  0.0114,  0.0137,  0.0171, -0.0260, -0.0600, -0.0210,
           0.0187],
         [-0.0039, -0.0480,  0.0250,  0.0482,  0.0345,  0.0295, -0.0453,
           0.0016, -0.0080, -0.0138,  0.0401, -0.0003,  0.0290,  0.0238,
           0.0807,  0.0181,  0.0196, -0.0168, -0.0008, -0.0282, -0.0430,
          -0.0064, -0.0137,  0.0314, -0.0011, -0.0011, -0.0265,  0.0301,
           0.0013, -0.0398,  0.0331, -0.0134, -0.0315,  0.0559,  0.0083,
           0.0105,  0.0319, -0.0267, -0.0087,  0.0029,  0.0318, -0.0150,
           0.0085, -0.0256, -0.0282, -0.0305, -0.0253,  0.0024,  0.0258,
          -0.0027, -0.0029, -0.0125,  0.0113, -0.0024, -0.0033, -0.0174,
           0.0096, -0.0284,  0.0347,  0.1069, -0.0543, -0.0258,  0.0036,
           0.0005],
         [ 0.0434,  0.0056, -0.0141,  0.0092, -0.0786, -0.0176, -0.0352,
          -0.0508, -0.0219,  0.0113,  0.0506, -0.0066,  0.0113, -0.0194,
           0.0565, -0.0103, -0.0102,  0.0038, -0.0593,  0.0133,  0.0187,
          -0.0486, -0.0413,  0.0115,  0.0285,  0.0119, -0.0227,  0.0383,
          -0.0440, -0.0037, -0.0299, -0.0312, -0.0275,  0.0711, -0.0049,
           0.0182,  0.0456, -0.0225, -0.0212,  0.0119,  0.0242, -0.0039,
          -0.0116, -0.0256, -0.0359,  0.0101,  0.0069, -0.0504, -0.0062,
           0.0264,  0.0083, -0.0268, -0.0339, -0.0195, -0.0023,  0.0256,
           0.0175, -0.0287, -0.0115,  0.0267, -0.0018, -0.0208, -0.0513,
           0.0553],
         [-0.0305,  0.0525, -0.0127, -0.0635, -0.0147, -0.0653, -0.0431,
           0.0274, -0.0037, -0.0150,  0.0383, -0.0516,  0.0458, -0.0270,
          -0.0005, -0.0168,  0.0378,  0.0612,  0.0004, -0.0292, -0.0409,
          -0.0046,  0.0528, -0.0026,  0.0004, -0.0215, -0.0011,  0.0014,
          -0.0219,  0.0273,  0.0295,  0.0107,  0.0298, -0.0159, -0.0308,
          -0.0092, -0.0109, -0.0074,  0.0250, -0.0262, -0.0372, -0.0280,
           0.0166, -0.0113,  0.0097, -0.0007,  0.0032, -0.0772, -0.0260,
           0.0200,  0.0142,  0.0386,  0.0122,  0.0437, -0.0385,  0.0345,
          -0.0523, -0.0094,  0.0264, -0.0192,  0.0211, -0.0039, -0.0199,
           0.0176]]], device='cuda:0', dtype=torch.bfloat16)

mask = torch.tensor([[1, 1, 1, 1]]).to(x.device).to(x.dtype)

emb_fp32_a = weighted_mean_pooling(x, mask, cast=True)
emb_bfp16_a = weighted_mean_pooling(x, mask, cast=False)
torch.sum(abs(emb_fp32_a - emb_bfp16_a))

tensor(0.0014, device='cuda:0', dtype=torch.bfloat16)

In [ ]:
emb_fp32_b = weighted_mean_pooling(x, mask, cast=True, method_b=True)
emb_bfp16_b = weighted_mean_pooling(x, mask, cast=False, method_b=True)
torch.sum(abs(emb_fp32_b - emb_bfp16_b))

tensor(0.0030, device='cuda:0', dtype=torch.bfloat16)

In [ ]:
assert (emb_fp32_a == emb_fp32_b).all()

In [ ]:
emb_fp16_a = weighted_mean_pooling(x, mask, cast=False, fp16=True)
emb_fp16_b = weighted_mean_pooling(x, mask, cast=False, fp16=True, method_b=True)
emb_fp16_a, emb_fp16_b

(tensor([[0., -0., 0., -0., -0., -0., -0., 0., -0., -0., 0., -0., 0., -0., 0., -0., 0., 0., -0., -0., -0., -0., -0., -0.,
          0., 0., -0., 0., -0., -0., 0., -0., -0., 0., -0., 0., 0., -0., 0., -0., 0., -0., 0., -0., -0., -0., -0., -0.,
          0., 0., 0., -0., 0., 0., -0., 0., -0., -0., 0., 0., -0., -0., -0., 0.]],
        device='cuda:0', dtype=torch.float16),
 tensor([[ 0.0040, -0.0020,  0.0003, -0.0048, -0.0152, -0.0251, -0.0475,  0.0075,
          -0.0127, -0.0010,  0.0487, -0.0060,  0.0381, -0.0133,  0.0346, -0.0090,
           0.0153,  0.0018, -0.0265, -0.0084, -0.0212, -0.0173, -0.0190, -0.0014,
           0.0067,  0.0024, -0.0186,  0.0231, -0.0158, -0.0063,  0.0091, -0.0012,
          -0.0163,  0.0415, -0.0137,  0.0086,  0.0259, -0.0098,  0.0084, -0.0046,
           0.0069, -0.0135,  0.0053, -0.0225, -0.0200, -0.0080, -0.0104, -0.0334,
           0.0018,  0.0282,  0.0196, -0.0007,  0.0030,  0.0037, -0.0214,  0.0141,
          -0.0003, -0.0138,  0.0158,  0.0328, -0.0151,

For BF16 method_a is better as it focuses on exponents not fractions.
For FP16 method_b is better as it focuses on exponents not digits.